# Использование сверточных нейросетей для выделения прекуссионной партии из музыкального трека

Авторы: Белозёров Михаил, Тескер Константин, Рыбаков Владислав, Бекбулатов Рамзан

#### Аннотация

В этой работе была реализована (попытка) решения задачи выделения барабанной партии из музыкального трека, не содержащего выделеной информации о прекуссионной партии. Были использованы методы обработки звукового потока, основаные на использовании сверточных нейросетей. Для ускорения вычислений и возможности применения линейных слоёв к данным в формате спектрограмм, имеющих три канала, были использованы векторные произведения для каждого нейрона линейного слоя. Полученые результаты показывают наличие недоработок в алгоритмах и достаточно большую погрешность при применении к тестовым данным. Однако полученные результаты демонстрируют применимость нейросетей к решению данного рода задач.

### Постановка задачи

Данная задача крайне актуальна в сфере обработки и производства музыки из-за малого количества разделеных треков и большой востребованности таких записей для репетиций и записи треков. Сложность данной задачи заключается в том, что выделение барабанной партии методом обрезания частоты сигнала не приносит желаемых результатов из-за огромной вариативности ударных инструментов и их звучания. 

В связи со спецификой задачи, при обработке данных необходимо учитывать не только данные с текущего момента но и учитывать информацию из "прошлого" и "будущего". Для этого в данной реализации входными данными являются данные из трех последовательных моментов времени. 

За основу при реализации были взяты результаты данной работы http://mac.citi.sinica.edu.tw/%7Eyang/pub/fan17dlm.pdf

### Данные

#### Входные данные

Для разработки нейросети мы искали подходящий датасет, который должен содержать как отдельные инструменты, которые мы хотим выделять, так и отдельные треки инструментов, на которых нейросеть должна была обучаться. На просторах интернета существует множество различных датасетов. 
iKala dataset состоит из 252 30-секундных сэмплов различных популярных песен, записанных 6 различными вокалистами. В этот датасет включены также набор текстов к каждому треку, жанр и прочая информация. Этот датасет можно получить по запросу у создателей, подписав лицензионное соглашение.
MedleyDB dataset состоит из 70 профессионально записанных треков. К каждому треку прилагается дополнительная  информация. Каждая композиция имеет набор треков инструментов, использованных при записи. Этот датасет тоже выдается по запросу, в котором надо подписать лицензионное соглашение.
Наш выбор пал на Demixing Secrets Dataset (DSD100), поскольку он распространяется бесплатно и без запросов. Помимо этого, он содержит в себе 100 различных профессионально записанных треков. К каждому треку прилагается отдельная дорожка каждого музыкального инструмента. Он разнообразен по жанрам и стилям. Этот датасет специально создан для конкурса MUS2016, в котором требовалось выполнить похожую на нашу задачу по выделению отдельного инструмента или голоса из композиции. Также этот датасет выделяет то, что для него существует специальная библиотека на MATLAB и Python, позволяющая работать с треками.

#### Предобработка данных

В качестве предобработки данных были выполнены следующие шаги:
1. Треки обрезались до 2 минут.
2. Поскольку наша задача состояла в выделении барабанной части, то мы объединили все инструменты, кроме барабанов в один, оставив барабанную часть отдельно. Таким образом для обучения сети у нас существует три файла: трек, трек без барабанов и только барабаны.

### Особенности реализации

#### VPNN

CVPNN

Для визуализации спектрограммы используется изображение в формате RGB что дает нам на входе нейросети трехканальный тензор.

WVPNN

Для построения входных данных нейросети используются три момента времени склеенные по последней оси, таки образом данные получают вид тензора с размерностью канала равной трем.

#### LOSS

Для рассчета функции потерь использовался  MSE-loss, рассчитвыющий разницу между магнитудным представлением изначальной барабанной партии и извлеченной нейросетью

Для уменьшения тяготения сети к склеиванию схожих ответов (барабанные партии часто похожи по ритму и/или темпу) в функцию ошибки добавлено слагаемое отвечаюзщее за поощерение разницы между различными треками.

### Особенности архитектуры сети

В процессе обучения предсказанная спектрограмма для перкуссионной партии $\tilde{Z}_1$ и музыкального спектра $\tilde{Z}_2$ слои WVPNN и CVPNN объединяются с функцией ошибки:
\begin{equation}
 L = ||\tilde{Z}_1 - Z_1||^2 + ||\tilde{Z}_2 - Z_2||^2
\end{equation}

После этого предсказанные спектрограммы $y_1$ и $y_2$ сглаживаются, и с помощью обратного преобразования Фурье получаются звуковые дорожки перкуссионной партии и обычной партии.

### Архитектура нейронной сети

* DNN слои
* 2 полносвязных слоя (для каждого источника)
* Нормализационный слой (Time-frequency masking layer)

![nn](images/model.png)

### Архитектура всей сети

![overall_model](images/overall.png)

### Тесты

Dataset был разделён на train и test в соотношении 3:2. Во время обучения нейросети стала явно видна сильная зависимость от параметров. Часто проявлялась проблема взрывающихся градиентов, и, как следствие - отсутствие сходимости. Результаты экспериментов проверялись на слух воспроизведением файлов и вычислением ошибки на тестовой части датасета. В большистве опробованных вариантов нейросеть выделяла из треков низкие частоты, которые включали в себя не только барабанные партии, но и другие инструменты: бас-гитара, электрогитара и, порой, вокал.

### Ссылки

[Music Signal Processing Using Vector Product Neural Networks. Z.C. Fan, T.S. Chan, Y.H. Yang, J.S. R. Jang
(Submitted on 29 Jun 2017)]

[Tohru Nitta. N-dimensional vector neuron. In Proc. IJCAI, pages 2–7, 2007.]

### Вклад в работу

* **Константин**: основная часть написания алгоритма
* **Михаил**: поиск и предобработка данных, помощь в написании алгоритма 
* **Владислав**: помощь в написании алгоритма, проведение эксперимента. 
* **Рамзан**: Проводил эксперименты, помогал отлаживать, устанваливал необходимое окружение